# Week4: Calibration，Single view metrology， Photometric image formation

This week, we build on our understanding of camera models by examining calibration and single-view metrology, learning how to determine camera parameters, address lens distortions, and infer scene geometry from a single image. We then turn to photometric image formation and reflectance models, exploring how light, surface properties, and material characteristics shape the intensity values captured by a camera. These combined insights set the stage for more advanced methods like photometric stereo and refined 3D reconstruction.

In this section, we will learn about:

- types of distortion caused by cameras
- how to find the intrinsic and extrinsic properties of a camera
- how to undistort images based off these properties

Ref: https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html

#### 1. Please complete the following coding tasks, and then proceed to answer the three questions.

#### 2. Provide written answers to the reflection questions in the Markdown cells below each question.

#### 3. Submit the final .ipynb file.

Pinhole cameras often distort images through radial and tangential distortions. Correcting these requires understanding the camera's intrinsic parameters (like focal length and optical centers) and extrinsic parameters (orientation and position). Intrinsic parameters help form a camera matrix, a 3x3 matrix crucial for correcting lens distortions. This matrix is unique to each camera and, once computed, can correct any image taken by that camera.

Let get started!

For camera calibration with OpenCV, we need images of a chessboard. Calibration requires matching 3D real world points to 2D image points. The latter, found where chessboard squares meet, is straightforward to identify. Assuming the chessboard is fixed on the XY plane (Z=0), we simplify to finding just the X,Y coordinates. These can be set as (0,0), (1,0), (2,0), etc., reflecting the chessboard squares' positions, with results scaled by square size. Without knowing the exact square size, we use relative square dimensions, referring to these 3D and 2D points as object and image points, respectively.

To detect chessboard patterns, use cv.findChessboardCorners(), specifying the grid size (e.g., 7x6 for this example). The function returns corner points and a boolean indicating pattern detection success.

In [1]:
import numpy as np
import cv2 as cv
import glob
import matplotlib.pyplot as plt

In [ ]:
# Hint: Use the appropriate termination criteria
# Criteria for corner refinement (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, max_iter, epsilon)
criteria = (... , ..., ...)

# Prepare object points for a 7x9 or 6x8 chessboard pattern
# Hint: Use np.zeros(...) and np.mgrid(...) to generate a grid of points.
# The shape depends on your actual chessboard size.
objp = ...
objp[:,:2] = ...  # Generate a grid of points in (x, y)

# Arrays to store object points and image points from all images
objpoints = []  # 3D points in real-world space
imgpoints = []  # 2D points in the image plane

# Hint: Provide the correct path to your folder containing chessboard images
# Files: './demo/camera_calibration/*.jpg'
images = glob.glob('...')

for fname in images:
    img = cv.imread(fname)
    
    # Convert to grayscale
    gray = ...
    
    # Attempt to find the chessboard corners
    # Hint: use cv.findChessboardCorners with the correct pattern size, e.g. (8,6)
    ret, corners = cv.findChessboardCorners(gray, (..., ...), None)

    if ret:
        # If corners are found, add object points and refined corner positions
        objpoints.append(objp)

        # Refine the corner positions
        corners2 = cv.cornerSubPix(gray, corners, (..., ...), (-1,-1), criteria)
        imgpoints.append(corners2)

        # Draw the corners on the image (cv.drawChessboardCorners)
        cv.drawChessboardCorners(img, (..., ...), corners2, ret)

        # Convert the image to RGB for matplotlib display
        img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)

        # Display the image with the detected corners
        plt.figure(figsize=(10, 6))
        plt.imshow(img_rgb)
        plt.title(f"Chessboard Corners Detected in {fname.split('/')[-1]}")
        plt.axis('off')
        plt.show()

# After processing, you can attempt camera calibration using cv.calibrateCamera
# (Not shown here, but after collecting objpoints and imgpoints, you can call:
# ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

plt.close('all')


## Calibration
Now that we have our object points and image points, we are ready to go for calibration. We can use the function, cv.calibrateCamera() which returns the camera matrix, distortion coefficients, rotation and translation vectors etc.



In [ ]:
# Fill you code here, by using: cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

## Undistortion
Now, we can take an image and undistort it. OpenCV comes with two methods for doing this. 

However first, we can refine the camera matrix based on a free scaling parameter using cv.getOptimalNewCameraMatrix(). If the scaling parameter alpha=0, it returns undistorted image with minimum unwanted pixels. So it may even remove some pixels at image corners. If alpha=1, all pixels are retained with some extra black images. This function also returns an image ROI which can be used to crop the result.

So, we take a new image (a.jpg in this case. That is the first image in this chapter)

In [ ]:
# Assume we have obtained camera matrix (mtx) and distortion coefficients (dist)
# from a previous calibration step. The student should have mtx, dist from calibrateCamera.

# Hint: Load the distorted image. Modify the path if needed.
img = cv.imread('...')  # e.g., './demo/a.jpg'
h, w = img.shape[:2]

# Hint: Use cv.getOptimalNewCameraMatrix with appropriate parameters.
# alpha parameter can be experimented with (0.0 to 1.0).
# This function returns a refined camera matrix and ROI.
newcameramtx, roi = ...

# Convert the image to RGB for matplotlib display
img_rgb = ...

plt.imshow(img_rgb)
plt.title('Raw Image (Distorted)')
plt.axis('off')
plt.show()
plt.close('all')

#######################################
# Undistortion
#######################################
# Using newcameramtx and dist to undistort the image.
# Hint: Use cv.undistort() with the original image, mtx, dist, and newcameramtx


undistorted_rgb = ...

plt.imshow(undistorted_rgb)
plt.title('Undistorted Image')
plt.axis('off')
plt.show()
plt.close('all')


### 1. Using <strong>cv.undistort()</strong>
This is the easiest way. Just call the function and use ROI obtained above to crop the result.

In [ ]:
# Assume we have:
# - img: the distorted input image
# - mtx: the camera matrix
# - dist: the distortion coefficients
# - newcameramtx: the optimized camera matrix
# - roi: the region of interest returned by cv.getOptimalNewCameraMatrix

# Undistort the image using cv.undistort
# Hint: cv.undistort(img, mtx, dist, None, newcameramtx)

# Convert to RGB for display
dst_rgb = ...

plt.imshow(dst_rgb)
plt.title('Undistorted Image')
plt.axis('off')
plt.show()
plt.close('all')


### 2. Using <strong>remapping</strong>
This way is a little bit more difficult. First, find a mapping function from the distorted image to the undistorted image. Then use the remap function.

In [ ]:
# Hint: Use cv.initUndistortRectifyMap to create the mapping

# Convert to RGB for display
dst_rgb = ...

plt.imshow(dst_rgb)
plt.title('Remapped Undistorted Image')
plt.axis('off')
plt.show()


## Re-projection Error
Re-projection error gives a good estimation of just how exact the found parameters are. The closer the re-projection error is to zero, the more accurate the parameters we found are. Given the intrinsic, distortion, rotation and translation matrices, we must first transform the object point to image point using cv.projectPoints(). Then, we can calculate the absolute norm between what we got with our transformation and the corner finding algorithm. To find the average error, we calculate the arithmetical mean of the errors calculated for all the calibration images.

In [ ]:
mean_error = 0

# Assume we have:
# objpoints: the list of 3D points in the real world space
# imgpoints: the list of 2D points in the image plane
# rvecs, tvecs: rotation and translation vectors from cv.calibrateCamera
# mtx: camera matrix
# dist: distortion coefficients

for i in range(len(objpoints)):
    # Hint: Use cv.projectPoints to project 3D points into the 2D image plane

    # Calculate the error between the detected image points (imgpoints) and the projected points (imgpoints2)
    # Use cv.norm with cv.NORM_L2 and divide by len(imgpoints2) to get average error per point.
    ...
# print("Total re-projection error:")


#### 1. What is the goal of camera calibration, and why do we need a sufficient number of correspondences between 3D points and their image projections?
(Write your answer here)


#### 2. In single-view metrology, how can perspective projection and vanishing lines help us infer scene structure from a single image?
(Write your answer here)


#### 3. How do reflectance models, such as Lambertian and specular models, affect the interpretation of image intensities in photometric image formation?
(Write your answer here)
